# 2 Собираем датасет транзакций

## Выделяем фичи справочников

Как мы увидели ранее, у на есть два справочника.

Каждый из справочников состоит их 2х полей: ключ и описание.

Попробуем выделить из описания дополнительные фичи.

### Справочник MCC

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 400)

In [17]:
tr_mcc_codes = pd.read_csv('kaggle_datasets/tr_mcc_codes.csv', delimiter=';')
tr_mcc_codes.sample(7, random_state=7)

,mcc_code,mcc_description
182,9399,"Правительственные услуги, нигде ранее не классифицируемые"
179,9211,"Судовые выплаты, включая алименты и детскую поддержку"
131,7011,"Отели, мотели, базы отдыха, сервисы бронирования"
24,4900,"Коммунальные услуги — электричество, газ, санитария, вода"
65,5541,Станции техобслуживания
104,5947,"Магазины открыток, подарков, новинок и сувениров"
32,5074,Оборудование для водопровода и отопительной системы


Видим, что описание достаточно подробное и сложносоставное. 

Оно может состоять из нескольких частей, разделенных тире (как коммунальные услуги) или запятыми (как судовые выплаты).

Сформируем фичи, выделяющие текст до этих символов как отдельную фичу.

In [25]:
tr_mcc_codes['mcc_full_description'] = tr_mcc_codes['mcc_description']


# формируем сокращенное описание MCC
def extract_text_before_delimiter(value):
    delimiters = [',', '—', '/']
    for delimiter in delimiters:
        if delimiter in value:
            value = value.split(delimiter)[0].strip()
    return value

for index, row in tr_mcc_codes.iterrows():
    tr_mcc_codes.at[index, 'mcc_description'] = extract_text_before_delimiter(row['mcc_description'])

# добавляем к mcc_description значение кода MCC, чтобы исключить дубликаты в этом поле
tr_mcc_codes['mcc_description'] = tr_mcc_codes['mcc_description'] + ' (' + tr_mcc_codes['mcc_code'].astype('str') + ')' 


# выделяем группу MCC
tr_mcc_codes['mcc_group'] = tr_mcc_codes['mcc_code'].astype('str').apply(lambda x: x[:2])


import os
os.makedirs('./datasets', exist_ok=True)

tr_mcc_codes.to_csv('datasets/tr_mcc_codes_featured.csv', index=False)
tr_mcc_codes.head(5)

,mcc_code,mcc_description,mcc_full_description,mcc_group
0,742,Ветеринарные услуги (742) (742),Ветеринарные услуги (742),74
1,1711,Генеральные подрядчики по вентиляции (1711) (1711),Генеральные подрядчики по вентиляции (1711),17
2,1731,Подрядчики по электричеству (1731) (1731),Подрядчики по электричеству (1731),17
3,1799,Подрядчики (1799) (1799),Подрядчики (1799),17
4,2741,Разнообразные издательства (2741) (2741),Разнообразные издательства (2741),27
